In [148]:
import os
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
random_state = 42
np.random.seed(42)

In [30]:
train_path = './data/train.csv'
test_path = './data/test.csv'
bus_bts_path = './data/bus_bts.csv'
submission_path = './data/submission_sample.csv'

In [31]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
bus_bts = pd.read_csv(bus_bts_path)
submission = pd.read_csv(submission_path)

In [5]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
test.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff
0,415423,2019-10-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,415424,2019-10-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
2,415425,2019-10-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,415426,2019-10-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
4,415427,2019-10-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
set(train.columns) - set(test.columns)

{'18~20_ride'}

In [41]:
data.head()

,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff
0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [57]:
bus_bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1


In [56]:
data['6~7_ride'].sum() + data['7~8_ride'].sum() + data['8~9_ride'].sum() + data['9~10_ride'].sum() + data['10~11_ride'].sum() + data['11~12_ride'].sum()

2438837.0

In [10]:
submission.head()

,id,18~20_ride
0,415423,0
1,415424,0
2,415425,0
3,415426,0
4,415427,0


In [11]:
y_train = train['18~20_ride']

In [12]:
data = pd.concat([train.drop('18~20_ride', axis=1), test]).reset_index(drop=True)

In [14]:
print(train.shape)
print(test.shape)
print(data.shape)

(415423, 21)
(228170, 20)
(643593, 20)


In [16]:
data.tail()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff
643588,643588,2019-10-16,32820000,시내,786,고산환승정류장(고산1리),33.30073,126.18044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
643589,643589,2019-10-16,32820000,시내,1080,애월고등학교,33.46262,126.33447,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
643590,643590,2019-10-16,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
643591,643591,2019-10-16,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
643592,643592,2019-10-16,32820000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0


In [25]:
# id는 필요 없으므로 제거
data = data.drop('id', axis=1)

In [29]:
data.isnull().sum()

date             0
bus_route_id     0
in_out           0
station_code     0
station_name     0
latitude         0
longitude        0
6~7_ride         0
7~8_ride         0
8~9_ride         0
9~10_ride        0
10~11_ride       0
11~12_ride       0
6~7_takeoff      0
7~8_takeoff      0
8~9_takeoff      0
9~10_takeoff     0
10~11_takeoff    0
11~12_takeoff    0
dtype: int64

In [58]:
# column 별 unique 값 갯수 확인
for col in data.columns:
    print(f'column : {col}')
    print(f'The number of unique : {data[col].nunique()}')
    print('\n')

column : date
The number of unique : 46


column : bus_route_id
The number of unique : 631


column : in_out
The number of unique : 2


column : station_code
The number of unique : 3601


column : station_name
The number of unique : 1971


column : latitude
The number of unique : 3310


column : longitude
The number of unique : 3481


column : 6~7_ride
The number of unique : 48


column : 7~8_ride
The number of unique : 83


column : 8~9_ride
The number of unique : 84


column : 9~10_ride
The number of unique : 71


column : 10~11_ride
The number of unique : 69


column : 11~12_ride
The number of unique : 77


column : 6~7_takeoff
The number of unique : 35


column : 7~8_takeoff
The number of unique : 56


column : 8~9_takeoff
The number of unique : 53


column : 9~10_takeoff
The number of unique : 51


column : 10~11_takeoff
The number of unique : 51


column : 11~12_takeoff
The number of unique : 58




In [60]:
# in_out One-Hot Encoding
data = pd.get_dummies(data, columns=['in_out'])

In [83]:
data['bus_route_id'] = data['bus_route_id'] / 10000

In [89]:
le = LabelEncoder().fit(data['station_name'])

In [96]:
data['station_name'] = le.transform(data['station_name'])

In [110]:
import datetime

In [125]:
data['year'] = data['date'].map(lambda x: x[0:4])

In [129]:
data['month'] = data['date'].map(lambda x: x[5:7])

In [127]:
data['day'] = data['date'].map(lambda x: x[-2:])

In [132]:
data = data.drop('date', axis=1)

In [133]:
data.head()

,bus_route_id,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,...,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,in_out_시내,in_out_시외,year,month,day
0,427.0,344,1489,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0,1,2019,09,01
1,427.0,357,1831,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0,1,2019,09,01
2,427.0,432,1413,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,1,2019,09,01
3,427.0,1579,1438,33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,...,0.0,0.0,0.0,0.0,0.0,1,0,2019,09,01
4,427.0,1646,1583,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1,0,2019,09,01


In [134]:
X_train = data[:len(train)]
X_test = data[len(train):].reset_index(drop=True)

In [135]:
print(X_train.shape)
print(X_test.shape)

(415423, 22)
(228170, 22)


In [136]:
X_train.tail()

,bus_route_id,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,...,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,in_out_시내,in_out_시외,year,month,day
415418,3282.0,1129,1852,33.41437,126.26336,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,2019,09,30
415419,3282.0,1564,1482,33.49946,126.51479,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,2019,09,30
415420,3282.0,2322,1887,33.23100,126.26273,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1,0,2019,09,30
415421,3282.0,3291,1097,33.46483,126.31870,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,2019,09,30
415422,3282.0,6115100,788,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,1,0,2019,09,30


In [137]:
X_test.tail()

,bus_route_id,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,...,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,in_out_시내,in_out_시외,year,month,day
228165,3282.0,786,88,33.30073,126.18044,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,1,0,2019,10,16
228166,3282.0,1080,1091,33.46262,126.33447,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,2019,10,16
228167,3282.0,1129,1852,33.41437,126.26336,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,2019,10,16
228168,3282.0,1564,1482,33.49946,126.51479,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,0,2019,10,16
228169,3282.0,6115100,788,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,1,0,2019,10,16


In [152]:
params = {
    "n_estimators": 300,
    "random_state": random_state,
    "n_jobs": -1,
}

In [153]:
rf = RandomForestClassifier(**params)

In [154]:
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

In [155]:
pred = rf.predict(X_test)

In [156]:
pred

array([3., 6., 0., ..., 0., 0., 0.])

In [157]:
submission['18~20_ride'] = pred

In [158]:
submission

,id,18~20_ride
0,415423,3.0
1,415424,6.0
2,415425,0.0
3,415426,42.0
4,415427,0.0
...,...,...
228165,643588,0.0
228166,643589,0.0
228167,643590,0.0
228168,643591,0.0


In [159]:
submission.to_csv('./data/submission_baseline_rf.csv', index=False)

In [160]:
pd.read_csv('./data/submission_baseline_rf.csv')

,id,18~20_ride
0,415423,3.0
1,415424,6.0
2,415425,0.0
3,415426,42.0
4,415427,0.0
...,...,...
228165,643588,0.0
228166,643589,0.0
228167,643590,0.0
228168,643591,0.0
